In [ ]:
# pip install td-ameritrade-python-api

In [ ]:
import pandas as pd
import numpy as np

from td.client import TDClient
import yfinance as yf

import datetime

pd.set_option('display.float_format', '{:.2f}'.format)

# Connect to TDA

In [ ]:
CONSUMER_KEY = # YOUR KEY
REDIRECT_URI = # YOUR REDIRECT URL
TD_ACCOUNT = # YOUR TDA ACCOUNT ID

# Create a new session, credentials path is optional.
TDSession = TDClient(
    client_id=CONSUMER_KEY,
    redirect_uri=REDIRECT_URI,
    credentials_path='YOUR_PATH.json'
)

# Login to the session
TDSession.login()

# TDA Portfolio

In [ ]:
positions = TDSession.get_accounts(account=TD_ACCOUNT, fields=['positions'])
df_positions = pd.DataFrame(positions['securitiesAccount']['positions'])
df_positions.head()

In [ ]:
df_portfolio = (
    pd.concat([df_positions.drop('instrument', axis=1), df_positions['instrument'].apply(pd.Series)], axis=1)
    .loc[lambda x: x['assetType'] == 'EQUITY']
    [['symbol', 'marketValue']]
)

portfolio_market_value = df_portfolio.marketValue.sum()

df_portfolio

In [ ]:
print(f"Total value: ${df_portfolio.marketValue.sum():,.2f}")

# S&P 500 only portfolio

## TDA Transactions

`BUY` transactions for quotes in current portfolio

In [ ]:
transactions = TDSession.get_transactions(account=TD_ACCOUNT, transaction_type='BUY_ONLY')

In [ ]:
df_buys = (
    pd.json_normalize(transactions)
    .loc[lambda x: x['transactionItem.instrument.symbol'].isin(df_portfolio.symbol)]
    .rename(columns={
        'netAmount': 'amount',
        'transactionDate': 'dt'
    })
    [['dt', 'amount']]
    .assign(
        dt = lambda x: pd.to_datetime(x['dt']).dt.date,
        amount = lambda x: -x['amount']
    )
    .groupby(['dt'], as_index=False)
    ['amount']
    .sum()
)

df_buys

## S&P 500 price history

S&P 500 price history at the same dates as in transactions in previous step

In [ ]:
sp500_etf = 'VOO'

start = df_buys['dt'].min()
end = df_buys['dt'].max() + datetime.timedelta(days=1) # Include the last day
print(start, end)

In [ ]:
df_sp500 = yf.download(sp500_etf, 
                      start=start, 
                      end=end, 
                      progress=False)
df_sp500.head()

In [ ]:
df_sp500_clean = (
    df_sp500
    .reset_index()
    .rename(columns={
        'Date': 'dt',
        'Close': 'sp500_price'
    })
    [['dt', 'sp500_price']]
    .assign(
        dt = lambda x: pd.to_datetime(x['dt']).dt.date
    )
)

df_sp500_clean.head()

Current S&P 500 price

In [ ]:
df_buys_w_sp500 = (
    df_buys
    .merge(df_sp500_clean, how='left', on='dt')
    .assign(
        sp500_cnt = lambda x: x['amount'] / x['sp500_price']
    )
)

df_buys_w_sp500

In [ ]:
sp500_current = yf.Ticker(sp500_etf).history(period='1d')['Close'][0]
sp500_market_value = df_buys_w_sp500.sp500_cnt.sum() * sp500_current

print(f'Current {sp500_etf} price is ${sp500_current}')

# Compare 2 portfolios

In [ ]:
def portfolio_summary(name, open_balance, market_value):
    print(f"{name}\n")
    print(f"Open Balance: ${open_balance:,.2f}")
    print(f"Market value: ${market_value:,.2f}")
    print(f"Profit: ${market_value - open_balance:,.2f} ({(market_value - open_balance) / open_balance:,.2%})")

open_bal = df_buys_w_sp500.amount.sum()

In [ ]:
portfolio_summary('S&P 500 Portfolio only', open_bal, sp500_market_value)

In [ ]:
portfolio_summary('Current Portfolio', open_bal, portfolio_market_value)